In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from collections import Counter
import time
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('maxent_treebank_pos_tagger')
from lyricsgenius import Genius
import os
import dotenv
from dotenv import load_dotenv


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\savan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\savan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\savan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\savan\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


In [27]:
TOKEN = os.getenv('GeniusID')
load_dotenv()

genius = Genius(TOKEN)
genius.verbose = False # Turn off status messages
genius.remove_section_headers = False

x = "All Star"
y = "Smashmouth"

song = genius.search_song(x, y)

#print(song.lyrics)
lyrics = song.lyrics
lyricLine = lyrics.splitlines()
lyricList = []

for x in lyricLine:
    if "[" not in x:
        lyricList.append(x)
print(lyricList)

['Somebody once told me the world is gonna roll me', "I ain't the sharpest tool in the shed", 'She was looking kind of dumb with her finger and her thumb', 'In the shape of an "L" on her forehead', '', "Well, the years start comin' and they don't stop comin'", "Fed to the rules and I hit the ground runnin'", "Didn't make sense not to live for fun", 'Your brain gets smart but your head gets dumb', 'So much to do, so much to see', "So what's wrong with taking the backstreets?", "You'll never know if you don't go (W-w-wacko)", "You'll never shine if you don't glow", "Hey now, you're an all star", 'Get your game on, go play', "Hey now, you're a rock star", 'Get the show on, get paid', '(And all that glitters is gold)', "Only shootin' stars break the mold", "It's a cool place, and they say it gets colder", "You're bundled up now, wait 'til you get older", 'But the meteor men beg to differ', 'Judging by the hole in the satellite picture', "The ice we skate is gettin' pretty thin", "The water

In [32]:
Most_common_words =[
'the',	
'be',	
'to',	
'of',	
'and',	
'in',	
'that',
'have',
'it',	
'for',
'not',	
'on',	
'with',
'he',
'as',
'you',
'do',
'at',	
'this',	
'but',	
'his',	
'by',	
'from',	
'they',	
'we',	
'say',	
'her',
'she',	
'or',	
'an',	
'will',	
'my',	
'one',
'all',	
'would',
'there',
'their',
'what',	
'so',	
'up',	
'out',
'if',	
'about',	
'who',	
'get',	
'which',	
'go',	
'me',	
'when',	
'make',	
'can',	
'like',	
'time',	
'no',	
'just',	
'him',	
'know',	
'take',	
'people',	
'into',	
'year',
'your',
'good',
'some',
'could',
'them',
'see',
'other',
'than',	
'then',	
'look',	
'only',	
'come',	
'its',	
'over',	
'think',	
'also',	
'back',	
'after',	
'use',	
'two',	
'how',	
'our',	
'work',	
'first',	
'well',	
'way',	
'even',	
'new',
'want',
'because',
'any',
'these',	
'give',	
'day',	
'most',
'us',
'let']	

In [40]:
from nltk.tag import pos_tag
stopwords = nltk.corpus.stopwords.words('english')
from nltk.corpus import wordnet as wn
wnl = WordNetLemmatizer()
tokens = []

for x in lyricList:
    x = str(x).lower()
    x = str(x).replace("in'","ing")
    x = str(x).replace("'","")
    word_tokens = wordpunct_tokenize(x)
    for y in word_tokens:
        tokens.append(y)

print(tokens)

['somebody', 'once', 'told', 'me', 'the', 'world', 'is', 'gonna', 'roll', 'me', 'i', 'aingt', 'the', 'sharpest', 'tool', 'in', 'the', 'shed', 'she', 'was', 'looking', 'kind', 'of', 'dumb', 'with', 'her', 'finger', 'and', 'her', 'thumb', 'in', 'the', 'shape', 'of', 'an', '"', 'l', '"', 'on', 'her', 'forehead', 'well', ',', 'the', 'years', 'start', 'coming', 'and', 'they', 'dont', 'stop', 'coming', 'fed', 'to', 'the', 'rules', 'and', 'i', 'hit', 'the', 'ground', 'running', 'didnt', 'make', 'sense', 'not', 'to', 'live', 'for', 'fun', 'your', 'brain', 'gets', 'smart', 'but', 'your', 'head', 'gets', 'dumb', 'so', 'much', 'to', 'do', ',', 'so', 'much', 'to', 'see', 'so', 'whats', 'wrong', 'with', 'taking', 'the', 'backstreets', '?', 'youll', 'never', 'know', 'if', 'you', 'dont', 'go', '(', 'w', '-', 'w', '-', 'wacko', ')', 'youll', 'never', 'shine', 'if', 'you', 'dont', 'glow', 'hey', 'now', ',', 'youre', 'an', 'all', 'star', 'get', 'your', 'game', 'on', ',', 'go', 'play', 'hey', 'now', ',',

In [43]:
cleaned = []

for x in tokens:
    if len(x)>1:
        if x.isalpha():
            if x not in stopwords:
                if x not in Most_common_words:
                    cleaned.append(x)

print(cleaned)

['somebody', 'told', 'world', 'gonna', 'roll', 'aingt', 'sharpest', 'tool', 'shed', 'looking', 'kind', 'dumb', 'finger', 'thumb', 'shape', 'forehead', 'years', 'start', 'coming', 'dont', 'stop', 'coming', 'fed', 'rules', 'hit', 'ground', 'running', 'didnt', 'sense', 'live', 'fun', 'brain', 'gets', 'smart', 'head', 'gets', 'dumb', 'much', 'much', 'whats', 'wrong', 'taking', 'backstreets', 'youll', 'never', 'dont', 'wacko', 'youll', 'never', 'shine', 'dont', 'glow', 'hey', 'youre', 'star', 'game', 'play', 'hey', 'youre', 'rock', 'star', 'show', 'paid', 'glitters', 'gold', 'shooting', 'stars', 'break', 'mold', 'cool', 'place', 'gets', 'colder', 'youre', 'bundled', 'wait', 'til', 'older', 'meteor', 'men', 'beg', 'differ', 'judging', 'hole', 'satellite', 'picture', 'ice', 'skate', 'getting', 'pretty', 'thin', 'waters', 'getting', 'warm', 'might', 'swim', 'worlds', 'fire', 'bout', 'thats', 'ill', 'never', 'bored', 'hey', 'youre', 'star', 'game', 'play', 'hey', 'youre', 'rock', 'star', 'show'

In [58]:
lem_list = []

for x in cleaned:
    if len(wn.synsets(x)) >= 1:
        word = wn.synsets(x)[0]
        tmp = word.pos()
        lem = wnl.lemmatize(x, pos=tmp)
        lem_list.append(lem)

print(lem_list)
print(Counter(lem_list).most_common())

['somebody', 'tell', 'world', 'roll', 'sharp', 'tool', 'shed', 'looking', 'kind', 'dumb', 'finger', 'thumb', 'shape', 'forehead', 'year', 'start', 'coming', 'stop', 'coming', 'fed', 'rule', 'hit', 'ground', 'running', 'sense', 'live', 'fun', 'brain', 'get', 'smart', 'head', 'get', 'dumb', 'much', 'much', 'wrong', 'taking', 'never', 'wacko', 'never', 'shine', 'glow', 'star', 'game', 'play', 'rock', 'star', 'show', 'pay', 'glitter', 'gold', 'shooting', 'star', 'break', 'mold', 'cool', 'place', 'get', 'cold', 'bundle', 'wait', 'old', 'meteor', 'men', 'beg', 'differ', 'judging', 'hole', 'satellite', 'picture', 'ice', 'skate', 'getting', 'pretty', 'thin', 'water', 'getting', 'warm', 'might', 'swim', 'world', 'fire', 'bout', 'ill', 'never', 'bore', 'star', 'game', 'play', 'rock', 'star', 'show', 'pay', 'glitter', 'gold', 'shooting', 'star', 'break', 'mold', 'moon', 'wacko', 'wacko', 'moon', 'wacko', 'wacko', 'moon', 'moon', 'star', 'game', 'play', 'rock', 'star', 'show', 'pay', 'glitter', 'g

break n break Synset('look.n.02')
[Synset('look.n.02'), Synset('looking.n.02'), Synset('look.v.01'), Synset('look.v.02'), Synset('look.v.03'), Synset('search.v.02'), Synset('front.v.01'), Synset('attend.v.02'), Synset('look.v.07'), Synset('expect.v.03'), Synset('look.v.09'), Synset('count.v.08'), Synset('looking.s.01')]
